In [69]:
# Log on in app.airdata.com 
###########################

import re
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

options = webdriver.ChromeOptions()
# options.add_argument('--headless')

URL = 'https://app.airdata.com/flight/44beb41fe4fef44fa88f63c22a67ec0b/WEATHERWind_Map'

# Your initial setup here
driver = webdriver.Chrome(options=options)
driver.get('https://app.airdata.com/main?a=login')

# Locate login form fields using the By strategy
username_field = driver.find_element(By.NAME, 'email')
password_field = driver.find_element(By.NAME, 'password')

# Input credentials
username_field.send_keys('pietro.cicuta@gmail.com')
password_field.send_keys('***********')  # Replace with the current password
password_field.send_keys(Keys.RETURN)  # This will submit the form, alternatively find and click the submit button

# Wait for the authentication to complete, you might need explicit waits like WebDriverWait


In [6]:
# Download all wind data into CSV files
#######################################
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time
import re
import os

# # Initialize the web driver
# driver = webdriver.Chrome()

# Navigate to the page with the list of flight logs
driver.get('https://app.airdata.com/flight/05c7e9d257d58fcb1a6a55d3c717789c/WEATHERWind_Map')

# Your login code here

# Find all the flight logs based on their class names
flight_logs = driver.find_elements(By.CSS_SELECTOR, '.i_nor, .i_nor_selected')

# Iterate through each flight log
for flight_log in flight_logs:
    # Extract the name of the flight log and clean it up to use as a filename
    flight_name = flight_log.text
    sanitized_flight_name = re.sub('[^a-zA-Z0-9 \n\.]', '', flight_name).replace(" ", "_").replace(":", "")
    # Sanitize the flight name to create a valid file name
    sanitized_flight_name = re.sub(r'[^a-zA-Z0-9]', '_', flight_name)
  
    
    # Save to a CSV file in the wind_data folder
    csv_filename = f"wind_data/{sanitized_flight_name}.csv"
    if os.path.exists(csv_filename):
        continue
    
    ### click and wait for 10 sec
    # Click the flight log to load the wind data
    flight_log.click()
    
    # Assuming flight_log is the element from the previous page
    # WebDriverWait(driver, 10).until(EC.staleness_of(flight_log))
    # Wait for 10 seconds
    time.sleep(10)
    
    # ### click and wait until the checkbox is ticked (not working atm)
    # # Locate the checkbox associated with the flight log
    # checkbox = flight_log.find_element(By.XPATH, './preceding-sibling::td[@class="bulk"]/input[@type="checkbox"]')

    # # Click the flight log
    # flight_log.click()

    # # Wait for the checkbox to be ticked. A maximum wait of 10 seconds is set.
    # WebDriverWait(driver, 10).until(
    #     lambda driver: checkbox.get_attribute("checked") is not None
    # )

    
    # Get page source
    page_source = driver.page_source
    
    # Extract the relevant data
    pattern = r'Flight time: <B>(.*?)</B><BR>Altitude: <B>(.*?)</B><BR>Wind Direction: <B>(.*?)&deg;</B><BR>Wind Speed: <B>(.*?)</B>'
    matches = re.findall(pattern, page_source)
    
    # Convert matches to DataFrame
    df = pd.DataFrame(matches, columns=['Flight time', 'Altitude', 'Wind Direction', 'Wind Speed'])
    


    df.to_csv(csv_filename, index=False)
    
    print(f"CSV for '{flight_name}' created as '{csv_filename}'.")
        
    # Go back to the list (If the flight logs are on the same page, this step is not needed)
    # driver.back()
    
    # Update the list of flight logs
    flight_logs = driver.find_elements(By.CSS_SELECTOR, '.i_nor, .i_nor_selected')

# Close the driver
# driver.quit()



KeyboardInterrupt



In [70]:
# Download all media data into CSV files
#######################################
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time
import re
import os

# Navigate to the page with the list of flight logs
driver.get('https://app.airdata.com/flight/d4a4216e636ed751981ab4865614ddab/MEDIAMap')


# Find all the flight logs based on their class names
flight_logs = driver.find_elements(By.CSS_SELECTOR, '.i_nor, .i_nor_selected')

# Iterate through each flight log
for flight_log in flight_logs:
    # Extract the name of the flight log and clean it up to use as a filename
    flight_name = flight_log.text
    if flight_name=='OVERVIEW':
        continue
    sanitized_flight_name = re.sub('[^a-zA-Z0-9 \n\.]', '', flight_name).replace(" ", "_").replace(":", "")
    # Sanitize the flight name to create a valid file name
    sanitized_flight_name = re.sub(r'[^a-zA-Z0-9]', '_', flight_name)
  
    
    # Save to a CSV file in the wind_data folder
    csv_filename = f"media_data/{sanitized_flight_name}.csv"
    if os.path.exists(csv_filename):
        continue
    
    ### click and wait for 10 sec
    # Click the flight log to load the wind data
    flight_log.click()
    
    # # Assuming flight_log is the element from the previous page
    # # WebDriverWait(driver, 10).until(EC.staleness_of(flight_log))
    # # Wait for 10 seconds
    time.sleep(15)
    
    try:
        # Wait for the table to appear
        wait = WebDriverWait(driver, 20)
        table = wait.until(EC.presence_of_element_located((By.XPATH, '//table[@class="infoboxlabel"]')))
    except Exception as e:
        print(f"An error occurred 1: {e}")
        continue

    
    # Locate the table
    table = driver.find_element(By.XPATH, '//table[@class="infoboxlabel"]')

    # Extract headers
    headers = [header.text for header in table.find_elements(By.XPATH, './/tr[1]/td')]

    # Extract rows data
    # try:
    rows = table.find_elements(By.XPATH, './/tr[position()>1]')  # skip the header row
    data = []
    for row in rows:
        columns = row.find_elements(By.TAG_NAME, 'td')  # Corrected this line
        data.append([column.text for column in columns])

    # Create a DataFrame
    df = pd.DataFrame(data, columns=headers)

    # Save to CSV
    df.to_csv(csv_filename, index=False)
    
    print(f"CSV for '{flight_name}' created as '{csv_filename}'.")
    # except Exception as e:
    #     print(f"An error occurred 2: {e}")
    #     continue
        
    # Go back to the list (If the flight logs are on the same page, this step is not needed)
    # driver.back()
    
    # Update the list of flight logs
    flight_logs = driver.find_elements(By.CSS_SELECTOR, '.i_nor, .i_nor_selected')

# Close the driver
# driver.quit()

CSV for 'Jul 16th, 2023 09:15PM' created as 'media_data/Jul_16th__2023_09_15PM.csv'.
CSV for 'Jul 15th, 2023 03:22PM' created as 'media_data/Jul_15th__2023_03_22PM.csv'.
CSV for 'Jul 13th, 2023 06:54PM' created as 'media_data/Jul_13th__2023_06_54PM.csv'.
CSV for 'Jul 12th, 2023 01:11PM' created as 'media_data/Jul_12th__2023_01_11PM.csv'.
CSV for 'Jul 11th, 2023 06:26PM' created as 'media_data/Jul_11th__2023_06_26PM.csv'.
CSV for 'Jul 10th, 2023 06:54PM' created as 'media_data/Jul_10th__2023_06_54PM.csv'.
CSV for 'Jul 9th, 2023 07:02PM' created as 'media_data/Jul_9th__2023_07_02PM.csv'.
CSV for 'Jul 7th, 2023 04:56PM' created as 'media_data/Jul_7th__2023_04_56PM.csv'.
CSV for 'Jul 6th, 2023 05:41PM' created as 'media_data/Jul_6th__2023_05_41PM.csv'.
CSV for 'Jul 5th, 2023 06:49PM' created as 'media_data/Jul_5th__2023_06_49PM.csv'.
CSV for 'Jul 3rd, 2023 03:45PM' created as 'media_data/Jul_3rd__2023_03_45PM.csv'.
CSV for 'Jul 2nd, 2023 08:35PM' created as 'media_data/Jul_2nd__2023_08_35P